# Topic Modelling
Si richiede un'implementazione di un esercizio di Topic Modeling, utilizzando librerie open (come ad es. GenSim (https://radimrehurek.com/gensim/). Si richiede l'utilizzo di un corpus di almeno 1k documenti. Testare un algoritmo (ad esempio LDA) con più valori di k (num. di topics) e valutare la coerenza dei risultati, attraverso fine-tuning su parametri e pre-processing. Update: essendo che spesso i topic, per essere interpretabili, devono contenere content words, potete pensare di filtrare solamente i sostantivi in fase di preprocessing (cioè POS=noun).

- Topic modeling: partendo da un corpus abbastanza grande (almeno 1k documenti), provare ad estrarre topics
- Usando ad es. la libreria Gensim https://radimrehurek.com/gensim/


In [20]:
# Import dependencies
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from sklearn.datasets import fetch_20newsgroups
import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Downloading necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
filename = 'bbc-news-data.csv'
bbc_news_df = pd.read_csv(filename,sep='\t' )
bbc_news_df.head()



,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


### Preprocessing
La funzione `preprocess` viene utilizzata per preelaborare i dati di testo. Prende in input una frase ed esegue le seguenti operazioni:

1. Tokenizzazione: La frase viene suddivisa in singole parole.
2. Rimozione delle stopword: Ogni parola presente nell'elenco delle stopword viene rimossa dalla frase. Le stopword sono parole comuni che non hanno molto significato e vengono spesso rimosse nelle attività di elaborazione del linguaggio naturale.
3. Lemmatizzazione: Ogni parola della frase viene lemmatizzata. 
4. `Filtro` dei sostantivi WordNet: Tutte le parole che non hanno un corrispondente synset (insieme di sinonimi) in WordNet vengono eliminate. Questo perché utilizziamo WordNet per comprendere il contenuto semantico del testo, quindi ogni parola che non è presente in WordNet non ci è utile.
5. `Filtro` sul POS:  spesso i topic, per essere interpretabili, devono contenere content words, potete pensare di filtrare solamente i sostantivi in fase di preprocessing (cioè POS=noun).


Il punto 4 e 5 sono gestiti assieme nella funzione `filter_noun_contained_in_wd`


In [46]:
import string
from nltk.corpus import stopwords
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from statistics import mean
from itertools import product, starmap
from nltk.corpus import wordnet as wn
from collections import Counter



lemmatizer = WordNetLemmatizer()
additional_stopwords = ['\'s', '’']
punctuation = set(string.punctuation)
stopwordset = set(stopwords.words('english') + additional_stopwords)


def to_lower_case(words):
    return words.lower()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def lemmatize(words):
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word.lower(), pos='v')  # Specify the part-of-speech tag 'v' for verb
        lemmas.append(lemma)
    return lemmas


def remove_stopwords(words):
    return [word for word in words if word not in stopwordset]

def remove_punctuation(words):
    return [word for word in words if word not in punctuation]

def filter_noun_contained_in_wd(words):
    # Filter out words that are not nouns or are not contained in WordNet
    words = nltk.pos_tag(words)
    out =  [word[0] for word in words if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']]
    out = [word for word in out if len(wn.synsets(word)) > 0]
    return out


def preprocess_data(sentence):
    words = to_lower_case(sentence)
    words = tokenize(words)
    words = lemmatize(words)
    words = remove_stopwords(words)
    words = remove_punctuation(words)
    words = filter_noun_contained_in_wd(words)
    return words
  

  

sentence1 = 'He went to the bank to deposit my money asdasda'
sentence2 = 'The river bank is full of wild flowers'

context1_bow = preprocess_data(sentence1)
context2_bow = preprocess_data(sentence2)
print("Bag of words for context 1: ", context1_bow)
print("Bag of words for context 2: ", context2_bow)



Bag of words for context 1:  ['bank', 'deposit', 'money']
Bag of words for context 2:  ['river', 'bank', 'flower']


### Creazione del corpus e del dizionario

Costruire un corpus di documenti a partire dai dati pre-processati. 
- `Creazione del dizionario`:  Il dizionario mappa ogni parola univoca a un ID numerico. Nel caso dell'esempio, viene utilizzata la classe corpora.Dictionary del modulo corpora di Gensim per creare il dizionario id2word.
- ` Creazione del corpus`: Il corpus è una rappresentazione dei documenti in forma di bag-of-words, ovvero come una lista di tuple (ID parola, frequenza parola) per ciascun documento. Nel caso dell'esempio, viene utilizzata la funzione doc2bow() del dizionario id2word per convertire i token di ciascun articolo nel formato bag-of-words. 
- Creazione della `matrice di conteggio` delle parole: Creare una matrice di conteggio delle parole, che rappresenta la frequenza di ogni parola nel corpus. 

In [56]:
#extract 1000 sentences from the column 'content' of the dataframe into a list
bbc_news = bbc_news_df['content'].tolist()[:1000]

# Preprocess the data -> data is aldraedy tokenized (only nouns)
processed_bbc_news = [preprocess_data(sentence) for sentence in bbc_news]


In [58]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(processed_bbc_news)

# Create Corpus
texts = processed_bbc_news

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


#printing 50 words from the text corpus
corpus_example = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]
corpus_example[0][:50]

[('allegiances', 1),
 ('campbell', 1),
 ('caution', 1),
 ('chamber', 2),
 ('charge', 1),
 ('colleagues', 1),
 ('commons', 3),
 ('computers', 1),
 ('conventions', 1),
 ('debate', 2),
 ('deputies', 1),
 ('devices', 2),
 ('earpieces', 1),
 ('enforce', 1),
 ('house', 1),
 ('journalist', 1),
 ('martin', 2),
 ('members', 1),
 ('message', 1),
 ('michael', 1),
 ('mps', 2),
 ('mr', 1),
 ('order', 1),
 ('pager', 2),
 ('party', 1),
 ('phone', 2),
 ('prominence', 1),
 ('rebuke', 1),
 ('result', 1),
 ('reveal', 1),
 ('rule', 2),
 ('sound', 1),
 ('speaker', 2),
 ('use', 1),
 ('week', 1)]

### Addestramento del modello LDA

Utilizzare la matrice di conteggio delle parole per addestrare il modello LDA. Durante l'addestramento, il modello LDA stima le distribuzioni di topic per ogni documento e le distribuzioni di parole per ogni topic.

Proviamo a variare il termine "num_topics", cioè K.

In [59]:
# build LDA model for 10 topic
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='symmetric',
                                           per_word_topics=True,
                                           eta = 0.6)

In [61]:
from pprint import pprint

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"game" + 0.011*"time" + 0.010*"world" + 0.009*"play" + 0.008*"team" + '
  '0.007*"match" + 0.006*"season" + 0.006*"cup" + 0.006*"club" + '
  '0.006*"champion"'),
 (1,
  '0.026*"search" + 0.009*"software" + 0.008*"attack" + 0.008*"security" + '
  '0.006*"users" + 0.004*"result" + 0.004*"traffic" + 0.004*"engines" + '
  '0.004*"program" + 0.004*"virus"'),
 (2,
  '0.000*"presume" + 0.000*"mischief" + 0.000*"quite" + 0.000*"mansfield" + '
  '0.000*"deflation" + 0.000*"rail" + 0.000*"kraft" + 0.000*"invite" + '
  '0.000*"impress" + 0.000*"combination"'),
 (3,
  '0.016*"company" + 0.011*"year" + 0.010*"market" + 0.009*"firm" + '
  '0.008*"share" + 0.008*"bank" + 0.006*"growth" + 0.006*"price" + '
  '0.006*"sales" + 0.006*"mr"'),
 (4,
  '0.010*"glazer" + 0.010*"club" + 0.004*"board" + 0.004*"offer" + 0.003*"bid" '
  '+ 0.003*"proposal" + 0.002*"manchester" + 0.002*"foster" + 0.002*"detail" + '
  '0.002*"takeover"'),
 (5,
  '0.031*"film" + 0.011*"star" + 0.010*"year" + 0.008*"awa

Ogni tupla nell'output rappresenta un argomento identificato dal modello, con il suo indice numerico seguito da una lista di parole chiave associate all'argomento e i relativi pesi.

Ad esempio, nella prima tupla (0, '0.018*"game" + 0.011*"time" + ...'), l'argomento rappresentato dall'indice 0 è caratterizzato da parole chiave come "game", "time", "world", "play", ecc. I numeri che seguono ogni parola chiave indicano il peso associato a quella parola nell'argomento specifico.

L'output può essere interpretato nel seguente modo:

- `Ogni tupla `rappresenta un argomento identificato dal modello.
- Le `parole chiave` elencate nella tupla rappresentano le parole `più rilevanti` associate all'argomento.
- I `pesi` indicano l'`importanza` relativa delle parole `all'interno dell'argomento.` Più alto è il peso, più - rilevante è la parola per l'argomento.


### Analisi e visualizzazione dei topic

Visualizziamo ora quanto ottenuto dal modello LDA, come le distribuzioni di topic per i documenti e le distribuzioni di parole per i topic. Questo può includere l'identificazione dei topic più rilevanti, l'etichettatura dei topic e l'interpretazione dei risultati.
Utilizzerò  la libreria `pyLDAvis`, successivamente confronteremo gli output al variare di K.

In [63]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.144628 -0.055367       1        1  26.252094
6     -0.133571  0.062914       2        1  21.438488
3     -0.130742  0.082134       3        1  19.754526
0     -0.021987 -0.134255       4        1  13.864443
5      0.001494 -0.061136       5        1  12.936698
8      0.108217 -0.011261       6        1   2.884138
1      0.083199  0.054254       7        1   1.813415
4      0.079879  0.020854       8        1   0.631588
7      0.080781  0.021703       9        1   0.324355
2      0.077357  0.020160      10        1   0.100257, topic_info=            Term        Freq       Total Category  logprob  loglift
280         film  414.000000  414.000000  Default  30.0000  30.0000
169         game  535.000000  535.000000  Default  29.0000  29.0000
24         party  340.000000  340.000000  Default  28.0000  28.0000
47       company  439.000000  439.000000  Default  27.0000  27.0000
203   government  435.000000  435.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1692      butler    0.012406   10.699287  Topic10  -8.9828   0.1454
7186      walter    0.012406    8.787990  Topic10  -8.9828   0.3422
7778      barren    0.012406    4.703524  Topic10  -8.9828   0.9673
7049    pleasure    0.012406    8.538090  Topic10  -8.9828   0.3711
7748    concepts    0.012406    8.734400  Topic10  -8.9828   0.3483

[576 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1355      1  0.096237   access
1355      2  0.798771   access
1355      3  0.086614   access
1355      5  0.009624   access
185       1  0.112975  account
...     ...       ...      ...
263       5  0.197519    years
2712      1  0.134624  zealand
2712      3  0.044875  zealand
2712      4  0.044875  zealand
2712      6  0.717992  zealand

[1835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 4, 1, 6, 9, 2, 5, 8, 3])

Possiamo subito notare che diversi TOPIC overalappano. In particolare:
- il topic 2 sembra fare riferimento al mondo della tecnologia
- il topic 3 si riferisce al mondo business
- i topic 8,9 e 10
Quindi verifichiamo quali siano i termini che overlappano (dal grafico possiamo subito notare 'Company')

In [102]:
import re
# display overlapping words between topics

topic2_terms= lda_model.show_topics(num_topics=10, num_words=1000, log=False, formatted=True)[1]
topic3_terms= lda_model.show_topics(num_topics=10, num_words=1000, log=False, formatted=True)[2]

terms_topic1: list = re.findall(r'"([^"]*)"', topic2_terms[1])
terms_topic2: list = re.findall(r'"([^"]*)"', topic3_terms[1])
overlap_terms = set(terms_topic1).intersection(terms_topic2)

# print overlapping words between topic 2 and topic 3
# Print the overlapping terms
print("Overlap between topic 0 and topic 1:")
print(len(overlap_terms), overlap_terms)

Overlap between topic 0 and topic 1:
182 {'ten', 'secure', 'word', 'commercials', 'cardinal', 'bill', 'ferocity', 'bulgaria', 'invite', 'provide', 'christmas', 'client', 'christians', 'buck', 'amount', 'consultant', 'firm', 'robots', 'buy', 'impose', 'recommend', 'harper', 'saviour', 'suspicions', 'select', 'writers', 'contend', 'forward', 'approach', 'begin', 'rout', 'lie', 'engineer', 'fbi', 'spot', 'novelists', 'alienate', 'surround', 'felony', 'cod', 'twin', 'article', 'bell', 'milk', 'inspectors', 'canada', 'couple', 'britons', 'respond', 'relentless', 'periods', 'april', 'august', 'net', 'fall', 'drivers', 'outriders', 'roads', 'travel', 'victims', 'advice', 'ones', 'bet', 'rush', 'catch', 'hugo', 'tactics', 'operation', 'learn', 'host', 'california', 'netherlands', 'rowan', 'operate', 'choke', 'signal', 'respondents', 'holy', 'russell', 'injustice', 'contact', 'protect', 'exist', 'overhaul', 'scale', 'none', 'relationship', 'wish', 'topic', 'throw', 'criteria', 'tomorrow', 'moto

### Valutazione 

La `perplessità` è una misura comunemente utilizzata per valutare la qualità dei modelli di linguaggio, inclusi i modelli di topic come LSA. Funziona bene come metrica di valutazione anche per i modelli di LSA. In generale,` un punteggio di perplessità più basso indica una migliore performance del modello,` poiché significa che il modello è in grado di fare previsioni più accurate su dati di test separati. Pertanto, anche per valutare i modelli di LSA, la perplessità può essere un indicatore utile per misurare quanto bene il modello sia in grado di rappresentare il testo e fare previsioni coerenti.


` La coerenza dei topic` misura la coerenza semantica dei topic generati dal modello. Valuta quanto bene le parole principali di ciascun topic si allineano tra loro. Valori di coerenza più alti indicano topic più coerenti e interpretabili. Esistono diverse metriche di coerenza disponibili, come `c_v, u_mass e c_npmi,` che possono essere utilizzate per calcolare i punteggi di coerenza per i topic di LSA. 

Il punteggio di coerenza `c_v` può variare da 0 a 1, dove un punteggio più alto indica una maggiore coerenza dei topic. Un punteggio di coerenza più alto indica che le parole chiave all'interno di ciascun topic sono più coerenti e che i topic stessi sono più interpretabili.


Nota: dettaglio sulla metrica c_v
- Il modello di topic modeling viene addestrato utilizzando un corpus di testo.
- Per ogni topic generato dal modello, vengono estratte le parole chiave più rappresentative.
- Viene creata una matrice di co-occorrenza basata sulle parole chiave di ciascun topic. La matrice tiene traccia di quante volte le parole chiave compaiono insieme nei documenti - del corpus.
- Utilizzando la matrice di co-occorrenza, viene calcolato un punteggio di coerenza per ciascun topic. Il punteggio di coerenza tiene conto della distribuzione delle parole chiave della loro co-occorrenza nei documenti.
- Infine, i punteggi di coerenza per tutti i topic vengono combinati per ottenere un punteggio complessivo di coerenza per il modello.

In [103]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity : ', lda_model.log_perplexity(corpus)) 

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_bbc_news, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity :  -7.608441183665357

Coherence Score:  0.5461456665912069


Eseguo qualche test variando gli iper-parametri del modello LDA e ricalcolando queste metriche di qualità.
Oltre ai topic, gli iperparametri del modello sono:
- `Il parametro α` rappresenta la distribuzione a priori dei topic per ogni documento nel corpus. È un parametro che controlla la concentrazione della distribuzione dei topic all'interno di ciascun documento. 
- `Il parametro β` rappresenta la distribuzione a priori delle parole per ogni topic nel corpus. È un parametro che controlla la concentrazione delle parole all'interno di ciascun topic.

In [105]:
topics =        [5,15,25,30]
alpha_list =  [0.1,0.2,0.5,0.8]
beta_list =   [0.1,0.2,0.5,0.8]

In [106]:
def evaluateModel(n, alpha, beta):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=n, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           per_word_topics=True,
                                           eta = beta
                                           )
    perplexity = lda_model.log_perplexity(corpus)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_bbc_news, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda, perplexity

# loop over the number of topics
for topic in topics:
    for alpha in alpha_list:
        for beta in beta_list:
            coherence, perplexity = evaluateModel(topic, alpha, beta)
            print(f"n : {topic} ; alpha : {alpha} ; beta : {beta} ; Coherence : {coherence}; Perplexity : {perplexity}")

n : 5 ; alpha : 0.1 ; beta : 0.1 ; Coherence : 0.5149126385208275; Perplexity : -7.912543897985158
n : 5 ; alpha : 0.1 ; beta : 0.2 ; Coherence : 0.5101391621393587; Perplexity : -7.593121160430932
n : 5 ; alpha : 0.1 ; beta : 0.5 ; Coherence : 0.4886153707945128; Perplexity : -7.579847924921035
n : 5 ; alpha : 0.1 ; beta : 0.8 ; Coherence : 0.489502791260861; Perplexity : -7.641363792954393
n : 5 ; alpha : 0.2 ; beta : 0.1 ; Coherence : 0.49322436374921647; Perplexity : -7.899554711707095
n : 5 ; alpha : 0.2 ; beta : 0.2 ; Coherence : 0.5076769330692292; Perplexity : -7.5863163616158955
n : 5 ; alpha : 0.2 ; beta : 0.5 ; Coherence : 0.49592058362558306; Perplexity : -7.588316480134858
n : 5 ; alpha : 0.2 ; beta : 0.8 ; Coherence : 0.49097215886053025; Perplexity : -7.651202189383975
n : 5 ; alpha : 0.5 ; beta : 0.1 ; Coherence : 0.47359396103758816; Perplexity : -7.873361760582838
n : 5 ; alpha : 0.5 ; beta : 0.2 ; Coherence : 0.5014412060072614; Perplexity : -7.578350449611334
n : 5 

Il migliore modello è quello parametrizzato con Coerenza 0.696 e Perplessità -7.778
- `α`      -> 0.2
- `β`      -> 0.8
- `topics` -> 25

NOTA: per qualche ragione non riesco a plottare questo modello, type Error da parte di pyLDAVis che incontra un numero complesso che non riesce a serializzare.
Questo accade non appena provo a plottare un modello con +15 topic, α o β >0.5.

In [135]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=15, 
                                       random_state=100,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha=0.1,
                                       per_word_topics=True,
                                       eta = 0.8
                                       )

In [136]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.142734  0.079513       1        1  24.332240
6      0.142869 -0.116959       2        1  23.479612
10     0.145889 -0.013051       3        1  22.233461
8      0.048373  0.103236       4        1  15.604540
7     -0.014382 -0.004740       5        1   9.309647
14    -0.045651 -0.012504       6        1   3.480980
0     -0.051343  0.000645       7        1   0.660209
3     -0.046805 -0.004707       8        1   0.190292
12    -0.046007 -0.004508       9        1   0.111199
1     -0.045946 -0.004487      10        1   0.099636
2     -0.045946 -0.004487      11        1   0.099636
4     -0.045946 -0.004487      12        1   0.099636
5     -0.045946 -0.004487      13        1   0.099636
9     -0.045946 -0.004487      14        1   0.099636
13    -0.045946 -0.004487      15        1   0.099636, topic_info=              Term        Freq       Total Category  logprob  loglift
280           film  331.000000  331.000000  Default  30.0000  30.0000
169           game  518.000000  518.000000  Default  29.0000  29.0000
24           party  323.000000  323.000000  Default  28.0000  28.0000
47         company  416.000000  416.000000  Default  27.0000  27.0000
247          music  209.000000  209.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
7495            jr    0.012329    7.116497  Topic15  -8.9828   0.5532
5409           tag    0.012329   31.919323  Topic15  -8.9828  -0.9476
7338     dividends    0.012329    6.264670  Topic15  -8.9828   0.6807
577      shortlist    0.012329   11.791800  Topic15  -8.9828   0.0482
3859  preparations    0.012329   10.931650  Topic15  -8.9828   0.1239

[858 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
7607      1  0.432250  abandonment
7607      2  0.216125  abandonment
7607      3  0.216125  abandonment
7607      4  0.216125  abandonment
514       1  0.285927    abatement
...     ...       ...          ...
2712      1  0.027136      zealand
2712      2  0.027136      zealand
2712      3  0.027136      zealand
2712      4  0.922629      zealand
2712      5  0.027136      zealand

[2268 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 7, 11, 9, 8, 15, 1, 4, 13, 2, 3, 5, 6, 10, 14])